In [1]:
import tracemalloc, time
'''tracemalloc.start()
start_mem, _ = tracemalloc.get_traced_memory()
start_time = time.time()'''
f = open('GCF_000001405.39_GRCh38.p13_genomic.fna')
seq = {}
get_name = False
for line in f:
    if get_name == False:
        if line.startswith('>'):
            name = line.replace('>','').split()[0]
            if name != 'NC_000006.12':
                continue
            else:
                seq[name] = ''
                get_name = True
        else:
            continue
    else:
        if line.startswith('>'):
            break
        else:
            seq[name] += line.replace('\n','').strip()
            if len(seq[name]) > 200000:
                break
f.close()
S = seq['NC_000006.12'][100000:200000].lower()
'''end_time = time.time()
end_mem, _ = tracemalloc.get_traced_memory()
tracemalloc.stop()
print('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')'''

"end_time = time.time()\nend_mem, _ = tracemalloc.get_traced_memory()\ntracemalloc.stop()\nprint('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')"

In [2]:
dict_gen = {'a' : S.count('a') , 'c' : S.count('c') , 't' : S.count('t') , 'g': S.count('g') }
gen_prob = {'a' : S.count('a') / len(S), 'c' : S.count('c') / len(S), 't' : S.count('t') / len(S), 'g': S.count('g') / len(S)}
print(dict_gen, gen_prob, sep = '\n')

{'a': 31515, 'c': 20803, 't': 26886, 'g': 20796}
{'a': 0.31515, 'c': 0.20803, 't': 0.26886, 'g': 0.20796}


In [3]:
'''tracemalloc.start()
start_mem, _ = tracemalloc.get_traced_memory()
start_time = time.time()'''

#create trainsition matrix for order 1, 2
order_1_TM = {}
for i in dict_gen.keys():
    temp_dict_gen = dict_gen.copy()
    order_1_TM[i] = {}
    temp_dict_gen[i] -= 1
    for j in dict_gen.keys():
        order_1_TM[i][j] = temp_dict_gen[j] /(sum(dict_gen.values()) - 1)

order_2_TM = {}
for i in dict_gen.keys():
    for j in dict_gen.keys():
        order_2_TM['{}'.format(i + j)] = {}
        temp_dict_gen = dict_gen.copy()
        temp_dict_gen[i] -= 1
        temp_dict_gen[j] -= 1
        for k in dict_gen.keys():
            order_2_TM['{}'.format(i + j)][k] = temp_dict_gen[k] /(sum(dict_gen.values()) - 2)


'''end_time = time.time()
end_mem, _ = tracemalloc.get_traced_memory()
tracemalloc.stop()
print('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')'''

"end_time = time.time()\nend_mem, _ = tracemalloc.get_traced_memory()\ntracemalloc.stop()\nprint('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')"

In [4]:
# Markov models of order 0, 1 and 2
from math import log, exp, pow

def order_0():
    '''tracemalloc.start()
    start_mem, _ = tracemalloc.get_traced_memory()
    start_time = time.time()'''
    total = 0
    for i in dict_gen.keys():
        total += dict_gen[i] * log(gen_prob[i], 2)
    '''end_time = time.time()
    end_mem, _ = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')'''
    print('prob(log) :', total)
    return total

def order_1():
    '''tracemalloc.start()
    start_mem, _ = tracemalloc.get_traced_memory()
    start_time = time.time()'''
    total = log(gen_prob[S[0]], 2)
    for i in range(len(S)):
        if i > 0:
            total += log(order_1_TM[S[i-1]][S[i]], 2)
    '''end_time = time.time()
    end_mem, _ = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')'''
    print('prob(log) :', total)
    return total  

def order_2():
    '''tracemalloc.start()
    start_mem, _ = tracemalloc.get_traced_memory()
    start_time = time.time()'''
    total = 0
    for i in range(len(S)):
        if i == 0 or i == 1:
            total += log(gen_prob[S[i]], 2)
        else:
            total += log(order_2_TM['{}'.format(S[i - 2] + S[i - 1])][S[i]], 2)
    '''end_time = time.time()
    end_mem, _ = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')'''
    print('prob(log) :', total)
    return total 


print('order 0 :')
order_0()
print('\norder 1 :')
order_1()
print('\norder 2 :')
order_2()


order 0 :
prob(log) : -197688.94757656846

order 1 :
prob(log) : -197689.18170956918

order 2 :
prob(log) : -197689.38631291882


-197689.38631291882

In [5]:
from hmmlearn import hmm
import numpy as np

s = []
for i in S:
    for j in range(len(dict_gen)):
        if i == [k for k in dict_gen.keys()][j]:
            s.append(j)
s= np.array(s).reshape((-1, 1))

'''tracemalloc.start()
start_mem, _ = tracemalloc.get_traced_memory()
start_time = time.time()'''

n_states = 7
model = hmm.MultinomialHMM(n_components = n_states, n_iter = 1000, tol = 0.01)
model.fit(s)
print('prob(log) :', model.score(s) / log(2))

'''end_time = time.time()
end_mem, _ = tracemalloc.get_traced_memory()
tracemalloc.stop()
print('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')'''

prob(log) : -190477.2948489551


"end_time = time.time()\nend_mem, _ = tracemalloc.get_traced_memory()\ntracemalloc.stop()\nprint('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')"

In [8]:
'''tracemalloc.start()
start_mem, _ = tracemalloc.get_traced_memory()
start_time = time.time()'''

seq, output_state = model.sample(n_samples = 100000, random_state=None)
output_seq = ''
for i in seq.reshape((len(seq))):
    output_seq += [j for j in dict_gen.keys()][i]
print('output sequence :\n', output_seq, sep = '')

'''end_time = time.time()
end_mem, _ = tracemalloc.get_traced_memory()
tracemalloc.stop()
print('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')'''

output sequence :
...


"end_time = time.time()\nend_mem, _ = tracemalloc.get_traced_memory()\ntracemalloc.stop()\nprint('memory usage :', (end_mem - start_mem) / pow(10, 6), '(mb) ,run time :', end_time - start_time, '(s)')"